In [1]:
import sys
import os
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from logger_config import logger

from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
class ChildState(TypedDict):
    messages: Annotated[list, add_messages]

In [3]:
search_tool = TavilySearchResults(max_results=1)
tools = [search_tool]

In [4]:
llm = ChatGroq(model="llama-3.1-8b-instant")

llm_with_tools = llm.bind_tools(tools=tools)

In [5]:
def agent(state: ChildState):
    return {
        "messages": [llm_with_tools.invoke(state["messages"])], 
    }


In [6]:
def tools_router(state: ChildState):
    last_message = state["messages"][-1]

    if(hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0):
        return "tool_node"
    else: 
        return END


In [7]:
tool_node = ToolNode(tools=tools)

subgraph = StateGraph(ChildState)

subgraph.add_node("agent", agent)
subgraph.add_node("tool_node", tool_node)
subgraph.set_entry_point("agent")

subgraph.add_conditional_edges("agent", tools_router)
subgraph.add_edge("tool_node", "agent")

search_app = subgraph.compile()

In [9]:
search_app.invoke({"messages": [HumanMessage(content="How is the weather in Hyd?")]})

{'messages': [HumanMessage(content='How is the weather in Hyd?', additional_kwargs={}, response_metadata={}, id='5d188797-8adb-43c4-bb18-c7a1227fe040'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_repw', 'function': {'arguments': '{"query": "weather in Hyderabad today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 571, 'total_tokens': 592, 'completion_time': 0.028, 'prompt_time': 0.024628167, 'queue_time': -0.051786647, 'total_time': 0.052628167}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-98977ed0-7e39-4b45-b16d-2671473f43ef-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Hyderabad today'}, 'id': 'call_repw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 571, 'output_tokens': 21, 'total_tokens': 592}),
  ToolMessage(content='[{"

# Case 1: Shared Schema (Direct Embedding)

In [10]:
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, START, END
from langchain_core.messages import HumanMessage

In [11]:
# Define parent graph with the same schema
class ParentState(TypedDict):
    messages: Annotated[list, add_messages]


In [12]:
# Create parent graph
parent_graph = StateGraph(ParentState)

# Add the subgraph as a node
parent_graph.add_node("search_agent", search_app)

# Connect the flow
parent_graph.add_edge(START, "search_agent")
parent_graph.add_edge("search_agent", END)

# Compile parent graph
parent_app = parent_graph.compile()

In [16]:
# Run the parent graph
result = parent_app.invoke({"messages": [HumanMessage(content="How is the weather in hanamkonda, telangana?")]})
result

{'messages': [HumanMessage(content='How is the weather in hanamkonda, telangana?', additional_kwargs={}, response_metadata={}, id='07f49140-a6b1-4a08-ae02-6c950399532b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jnjh', 'function': {'arguments': '{"query": "hanamkonda telangana weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 583, 'total_tokens': 609, 'completion_time': 0.034666667, 'prompt_time': 0.025464977, 'queue_time': -0.061104507, 'total_time': 0.060131644}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f87dde32-3df2-4b0c-a098-aae89eea1ce7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'hanamkonda telangana weather today'}, 'id': 'call_jnjh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 583, 'output_tokens': 26, 'total_to

# Case 2: Different Schema (Invoke with Transformation)

In [17]:
from typing import TypedDict, Annotated, Dict
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage

In [18]:
# Define parent graph with different schema
class QueryState(TypedDict):
    query: str
    response: str

In [19]:
# Function to invoke subgraph
def search_agent(state: QueryState) -> Dict:
    # Transform from parent schema to subgraph schema
    subgraph_input = {
        "messages": [HumanMessage(content=state["query"])]
    }
    
    # Invoke the subgraph
    subgraph_result = search_app.invoke(subgraph_input)
    
    # Transform response back to parent schema
    assistant_message = subgraph_result["messages"][-1]
    return {"response": assistant_message.content}


In [20]:
# Create parent graph
parent_graph = StateGraph(QueryState)

# Add transformation node that invokes subgraph
parent_graph.add_node("search_agent", search_agent)

# Connect the flow
parent_graph.add_edge(START, "search_agent")
parent_graph.add_edge("search_agent", END)

# Compile parent graph
parent_app = parent_graph.compile()

In [21]:
# Run the parent graph
result = parent_app.invoke({"query": "How is the weather in KPHB?", "response": ""})
result

{'query': 'How is the weather in KPHB?',
 'response': 'The weather in KPHB is expected to be Light Drizzle with a maximum temperature of 34°C and a minimum of 25°C.'}